# TF1 Variable Reuse

#### Follows the article here: https://medium.com/@hideyuki/what-does-variable-reuse-mean-in-tensorflow-40e86535026b

In [1]:
import tensorflow as tf

<img src="https://miro.medium.com/max/1400/0*LLbCq-xvkHT7Wub7.png" width=400 />

In [2]:
def add_tax_and_shipping(price):
    """Add tax and shipping
    """    
    with tf.variable_scope("other_charge") as scope:
        tax = tf.get_variable(
            "tax", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(0.05)  # 5%
        )
        shipping = tf.get_variable(
            "shipping", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(10.0)  # $10
        )    
        total_price = price * (1.0 + tax) + shipping
    return total_price

In [3]:
book_price = add_tax_and_shipping(10.0)
init_op = tf.global_variables_initializer()  # Set up operator to assign all init values to variables
with tf.Session() as s:
    s.run(init_op)  # Actually assign initial value to variables
    price = s.run(book_price)
    print(f"Price of the book: {price:.2f}")

Price of the book: 20.50


**Now we try to add another item - sketch pad...**

In [4]:
try:
    sketchpad_price = add_tax_and_shipping(20.0)
    with tf.Session() as s:
        price = s.run(sketchpad_price)
        print(f"Price of the sketchpad: {price:.2f}")
except ValueError as ex:
    print(ex)

Variable other_charge/tax already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/mnt/space/miniconda3/envs/py37_tf1/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/mnt/space/miniconda3/envs/py37_tf1/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/mnt/space/miniconda3/envs/py37_tf1/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/mnt/space/miniconda3/envs/py37_tf1/lib/python3.7/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/mnt/space/miniconda3/envs/py37_tf1/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py", line 794, in _apply_op_helper
    

In [5]:
tf.reset_default_graph()  # To clean up the TF state before going to the next example...

**Notice exception:**

`Variable other_charge/tax already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:`

The reason you are getting this error is because TensorFlow prohibits calling the get_variable function for the same variable again by default.

How can you work around this?

Well, TensorFlow has a special keyword called `reuse`.

You specify the reuse parameter in the `variable_scope` where a variable is defined (`tf.variable_scope()`). 

In [6]:
try:
    with tf.variable_scope(
        "other_charge", 
        reuse=True  # <----------------- This line causes an exception to be thrown on the next line!
    ) as scope:
        tax = tf.get_variable(
            "tax", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(0.05)
        )
        shipping = tf.get_variable(
            "shipping", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(10.0)
        )
except ValueError as ex:
    print(ex)

Variable other_charge/tax does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?


**So in order to use `reuse=True`, the variable has to already exist.**

In [7]:
tf.reset_default_graph()  # To clean up the TF state before going to the next example...

### Approach 1: `tf.AUTO_REUSE`

In [8]:
def add_tax_and_shipping(price):
    with tf.variable_scope(
        "other_charge", 
        reuse=tf.AUTO_REUSE  # <----------------- Use tf.AUTO_REUSE
    ) as scope:
        tax = tf.get_variable(
            "tax", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(0.05)
        )
        shipping = tf.get_variable(
            "shipping", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(10.0)
        )
        total_price = price * (1.0 + tax) + shipping
    return total_price

In [9]:
book_price = add_tax_and_shipping(10.0)
sketchpad_price = add_tax_and_shipping(20.0)

init_op = tf.global_variables_initializer()

with tf.Session() as s:
    s.run(init_op)
    price = s.run(book_price)
    print(f"Price of the book: {price:.2f}")
    price = s.run(sketchpad_price)
    print(f"Price of the sketchpad: {price:.2f}")   


# Try to double tax!
print("Doubling tax...")
graph = tf.get_default_graph()
tax = graph.get_tensor_by_name("other_charge/tax:0")
double_tax = tf.assign(tax, tax * 2.0)

with tf.Session() as s:
    s.run(init_op)
    s.run(double_tax)
    price = s.run(book_price)
    print(f"Price of the book: {price:.2f}")
    price = s.run(sketchpad_price)
    print(f"Price of the sketchpad: {price:.2f}")  

Price of the book: 20.50
Price of the sketchpad: 31.00
Doubling tax...
Price of the book: 21.00
Price of the sketchpad: 32.00


In [10]:
tf.reset_default_graph()  # To clean up the TF state before going to the next example...

---

### ⚠️ Be careful with `AUTO_REUSE`!

`tf.AUTO_REUSE` is convenient, but you also have to be careful not to inadvertently share (reuse) variables. For example, this CIFAR-10 script defines a convolutional layer using a function below:

```python
def single_conv_block(
    tens, 
    block_name, 
    input_filters, 
    output_filters, 
    kernel_size=3, 
    padding="SAME", 
    strides=1,
    is_training=True
):
    scope_name = block_name + "_cv"
    with tf.variable_scope(
        scope_name,
        # reuse=tf.AUTO_REUSE  # <-- This here.
    ) as scope:
        weights = tf.get_variable(
            "weights", 
            shape=[kernel_size, kernel_size, input_filters, output_filters],
            dtype=tf.float32,
            initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0)
        )
        bias = tf.get_variable(
            "bias", 
            shape=[output_filters], 
            dtype=tf.float32
        )
        z = tf.add(
            tf.nn.conv2d(tens, weights, strides=[1, strides, strides, 1], padding=padding), 
            bias
        )
        bn = tf.layers.batch_normalization(z, training=is_training)
        activation = tf.nn.relu(bn, name=scope.name)
    return activation
```

If one 
1. forgets to use different `block_name` and also 
2. sets reuse to `tf.AUTO_REUSE`, 

multiple convolutional layers will **end up using the same weights**, which is clearly not what one wants!!!

---

### Approach 2: Explicit Control

If you want to explicitly control reuse of variable, you can explicitly say `reuse=True` **the second time you want to use the variable**.

In [11]:
def add_tax_and_shipping(
    price, 
    my_reuse=False  # <-- HERE.
):
    with tf.variable_scope(
        "other_charge", 
        reuse=my_reuse  # <-- HERE.
    ) as scope:
        tax = tf.get_variable(
            "tax", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(0.05)
        )
        shipping = tf.get_variable(
            "shipping", 
            [], 
            dtype=tf.float32,
            initializer=tf.constant_initializer(10.0)
        )
        total_price = price * (1.0 + tax) + shipping
    return total_price

In [12]:
book_price = add_tax_and_shipping(10.0)
sketchpad_price = add_tax_and_shipping(20.0, my_reuse=True)  # <-- SET HERE (2nd use onward).

init_op = tf.global_variables_initializer()

with tf.Session() as s:
    s.run(init_op)
    price = s.run(book_price)
    print(f"Price of the book: {price:.2f}")
    price = s.run(sketchpad_price)
    print(f"Price of the sketchpad: {price:.2f}")   

Price of the book: 20.50
Price of the sketchpad: 31.00
